# SPECT Dataset
By:Charu Saxena(A20378393)

# DATA DESCRIPTION

The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images.A SPECT scan of the heart is a noninvasive nuclear imaging test. It uses radioactive tracers that are injected into the blood to produce pictures of your heart. Doctors use SPECT to diagnose coronary artery disease. SPECT can show how well blood is flowing to the heart and how well the heart is working.It helps to find out  a person has coronary artery disease, had a heart attack or at a risk of it or has areas of scar tissue exist.

Each of the patients is classified into two categories: normal and abnormal. 
The database of 267 SPECT image sets (patients) was processed to extract features that summarize the original SPECT images.  
As a result, 44 continuous feature pattern was created for each patient.
The pattern was further processed to obtain 22 binary feature patterns.
The CLIP3 algorithm was used to generate classification rules from these patterns.The CLIP3 algorithm generated rules that were 84.0% accurate (as compared with cardilogists' diagnoses). 

     Number of Instances: 267
     Number of Attributes: 23 (22 binary + 1 binary class)
     Attribute Information:
   1.  OVERALL_DIAGNOSIS: 0,1 (class attribute, binary)
   2.  F1:  0,1 (the partial diagnosis 1, binary)
   3.  F2:  0,1 (the partial diagnosis 2, binary)
   4.  F3:  0,1 (the partial diagnosis 3, binary)
   5.  F4:  0,1 (the partial diagnosis 4, binary)
   6.  F5:  0,1 (the partial diagnosis 5, binary)
   7.  F6:  0,1 (the partial diagnosis 6, binary)
   8.  F7:  0,1 (the partial diagnosis 7, binary)
   9.  F8:  0,1 (the partial diagnosis 8, binary)
   10. F9:  0,1 (the partial diagnosis 9, binary)
   11. F10: 0,1 (the partial diagnosis 10, binary)
   12. F11: 0,1 (the partial diagnosis 11, binary)
   13. F12: 0,1 (the partial diagnosis 12, binary)
   14. F13: 0,1 (the partial diagnosis 13, binary)
   15. F14: 0,1 (the partial diagnosis 14, binary)
   16. F15: 0,1 (the partial diagnosis 15, binary)
   17. F16: 0,1 (the partial diagnosis 16, binary)
   18. F17: 0,1 (the partial diagnosis 17, binary)
   19. F18: 0,1 (the partial diagnosis 18, binary)
   20. F19: 0,1 (the partial diagnosis 19, binary)
   21. F20: 0,1 (the partial diagnosis 20, binary)
   22. F21: 0,1 (the partial diagnosis 21, binary)
   23. F22: 0,1 (the partial diagnosis 22, binary)
   
   
           dataset is divided into:
            a) training data ("SPECT.train" 80 instances)
            b) testing data ("SPECT.test" 187 instances)
    
            Missing Attribute Values: None

             Class Distribution:
              entire data
                Class		# examples
                   0		  55
                   1		  212
               training dataset
                 Class		# examples
                    0         40
                    1		  40
               testing dataset
                  Class		# examples
                    0		  15
                    1		  172

In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
from sklearn import naive_bayes
from sklearn.naive_bayes import BernoulliNB

# Load Dataset


In [2]:
train_data = pd.read_csv ("/Users/charusaxena/desktop/assignment2_data/SPECT-train.csv")
test_data = pd.read_csv ("/Users/charusaxena/desktop/assignment2_data/SPECT-test.csv")


In [3]:
from sklearn.model_selection import train_test_split
y = train_data['f23']
features = train_data
del features['f23']


In [4]:
lm = BernoulliNB()
model = lm.fit(features,y)
y_test = test_data['f23']
X_test = test_data
del X_test['f23']


In [5]:
predict = lm.predict_proba(X_test)
#predict
#print ("accuracy %2f" % model.score(X_test,y_test))

In [6]:
list_of_feature1no_prob = np.exp(lm.feature_log_prob_[0])
list_of_feature1yes_prob = np.exp(lm.feature_log_prob_[1])

print(list_of_feature1yes_prob)
list_of_feature0yes_prob = 1-list_of_feature1yes_prob
list_of_feature0no_prob = 1-list_of_feature1no_prob
list_of_feature0yes_prob

class_prob = np.exp(lm.class_log_prior_)



[ 0.71428571  0.39285714  0.28571429  0.42857143  0.35714286  0.32142857
  0.14285714  0.46428571  0.32142857  0.25        0.32142857  0.28571429
  0.46428571  0.57142857  0.28571429  0.14285714  0.35714286  0.21428571
  0.25        0.17857143  0.32142857  0.46428571]


In [77]:
#evidence/odds for both feature value 1 and 0 thus taking p(x=1/yes)/p(x=1/no) and p(x=0/yes)/p(x=0/no)
odds_feature0 = [a/b for a,b in zip(list_of_feature0yes_prob,list_of_feature0no_prob)]
odds_feature1 = [a/b for a,b in zip(list_of_feature1yes_prob,list_of_feature1no_prob)]

ods_class_prior = class_prob[1]/class_prob[0]
#adding class prior to both feature value 1 and feature value 0 odds/evidence p(+/-)

odds_feature0.append(ods_class_prior)#p(x=0/yes)/p(x=0/no)... p(+/-)
odds_feature1.append(ods_class_prior)#p(x=1/yes)/p(x=1/no).... p(+/-)
#odds_feature1,odds_feature0



In [170]:
def Total_evidence(object,class_prior):
    positive_evidence=[]
    negative_evidence=[]
    if np.log(class_prior)>0:
        positive_evidence.append(np.log(class_prior)) # add log(p(yes)/p(no)) if p(yes)/p(no) >1
    else:
        negative_evidence.append(np.log(class_prior))
    for i,val in enumerate(object):
        if val==1:
            if np.log(odds_feature1[i])>0:
                positive_evidence.append(np.log(odds_feature1[i]))#add log(p(x=0/yes)/p(x=0/no)) if p(x=0/yes)/p(x=0/no)>1
            else:
                negative_evidence.append(np.log(odds_feature1[i]))
        else:
            if np.log(odds_feature0[i])>0:
                positive_evidence.append(np.log(odds_feature0[i]))
            else:
                negative_evidence.append(np.log(odds_feature0[i]))
    
    total_positive_evidence=sum(positive_evidence)
    total_negative_evidence=sum(negative_evidence)
    return total_positive_evidence, total_negative_evidence

def evidence(object,class_prior):##also check if there is any previous value greater than the current postitve for a 
    #instnce
    positive_evidence=[]
    negative_evidence=[]
    
    for i,val in enumerate(object):
        if val==1:
            if np.log(odds_feature1[i])>0:
                positive_evidence.append(np.log(odds_feature1[i]))#add log(p(x=0/yes)/p(x=0/no)) if p(x=0/yes)/p(x=0/no)>1
            else:
                negative_evidence.append(np.log(odds_feature1[i]))
        else:
            if np.log(odds_feature0[i])>0:
                positive_evidence.append(np.log(odds_feature0[i]))
            else:
                negative_evidence.append(np.log(odds_feature0[i]))
   
    return positive_evidence, negative_evidence
 
    
def top_features(object,pos,neg):
    evidence=[]
    for i,val in enumerate(object):
        if val==1:
            
                evidence.append(np.log(odds_feature1[i]))#add log(p(x=0/yes)/p(x=0/no)) if p(x=0/yes)/p(x=0/no)>1
        else:
                evidence.append(np.log(odds_feature0[i]))
    
    evidence1=evidence
    Top_neg_index = np.argsort(evidence)
    
    #Top_pos_index = evidence.argsort()[::-1][:n]
    if neg>3:
        idx = 3
    else:
        idx=neg
        
    if pos>3:
        idx1 = 3
    else:
        idx1=pos  
    return Top_neg_index[:idx],Top_neg_index[-idx1:],evidence1



# 1. To find most positive object wrt to prob

    a) the total positive log-evidence,
    b) the total negative log-evidence, 
    c) probability distribution, 
    d) top 3 features values that contribute most to the positive evidence, 
    e) top 3 feature values that contribute the most to the negative evidence

In [178]:
#print("most positive object wrt to nprob is:",yesclass.index(max(yesclass)),"and value is :",max(yesclass))
most_postive_object = test_data.iloc[(np.argmax(predict[:,1]))]
print("most positive object wrt prob:\n",most_postive_object )

a = Total_evidence(most_postive_object,ods_class_prior)

print("\ntotal positive evidence:",a[0])
print("\ntotal negative evidence:",a[1])
print("\nclass distributuion",(predict[np.argmax(predict[:,1])]))


pos_evi=evidence(most_postive_object,ods_class_prior)[0]
neg_evi=evidence(most_postive_object,ods_class_prior)[1]

#print("\n top features",top_features(most_postive_object,len(pos_evi),len(neg_evi)))
top_features(most_postive_object,len(pos_evi),len(neg_evi))
indexNeg= top_features(most_postive_object,len(pos_evi),len(neg_evi))[0]
indexPos = top_features(most_postive_object,len(pos_evi),len(neg_evi))[1]

columns = X_test.columns

print("\n\ntop negative features are:",columns[indexNeg],"\ntop positive features",columns[indexPos])

#print("\n\n",evidence(most_postive_object,ods_class_prior))
#top_features(most_postive_object,len(pos_evi),len(neg_evi))[2]


most positive object wrt prob:
 f1     1
f2     1
f3     1
f4     1
f5     1
f6     1
f7     1
f8     1
f9     1
f10    1
f11    1
f12    0
f13    1
f14    1
f15    1
f16    1
f17    1
f18    1
f19    1
f20    1
f21    1
f22    1
Name: 158, dtype: int64

total positive evidence: 14.4112139432

total negative evidence: -1.05297100771

class distributuion [  1.57974911e-06   9.99998420e-01]


top negative features are: Index(['f20', 'f12'], dtype='object') 
top positive features Index(['f22', 'f17', 'f19'], dtype='object')


yes-1 and no-0

# 2.To find most negative object wrt probabilties

    a) the total positive log-evidence,
    b) the total negative log-evidence, 
    c) probability distribution, 
    d) top 3 features values that contribute most to the positive evidence, 
    e) top 3 feature values that contribute the most to the negative evidence

In [180]:
most_negative_object = test_data.iloc[(np.argmax(predict[:,0]))]

print("most negative object wrt prob:\n",most_negative_object )
for i,val in enumerate(most_negative_object):
    if val==1:
            print(i)

b= Total_evidence(most_negative_object,ods_class_prior)

print("\ntotal positive evidence:",b[0])
print("\ntotal negative evidence:",b[1])
print("\nclass distributuion",(predict[np.argmax(predict[:,0])]))


pos_evi=evidence(most_negative_object,ods_class_prior)[0]
neg_evi=evidence(most_negative_object,ods_class_prior)[1]

#print("\n top features",top_features(most_postive_object,len(pos_evi),len(neg_evi)))
top_features(most_negative_object,len(pos_evi),len(neg_evi))
indexNeg= top_features(most_negative_object,len(pos_evi),len(neg_evi))[0]
indexPos = top_features(most_negative_object,len(pos_evi),len(neg_evi))[1]

columns = X_test.columns

print("\n\ntop negative features are:",columns[indexNeg],"\ntop positive features",columns[indexPos])

most negative object wrt prob:
 f1     0
f2     0
f3     0
f4     0
f5     0
f6     0
f7     0
f8     0
f9     0
f10    0
f11    0
f12    0
f13    0
f14    0
f15    0
f16    0
f17    0
f18    0
f19    0
f20    0
f21    0
f22    0
Name: 172, dtype: int64

total positive evidence: 0.0444517625708

total negative evidence: -5.79641857806

class distributuion [ 0.99683353  0.00316647]


top negative features are: Index(['f1', 'f14', 'f22'], dtype='object') 
top positive features Index(['f20'], dtype='object')


# 3. To find object with highest positive evidence



    a) the total positive log-evidence,
    b) the total negative log-evidence, 
    c) probability distribution, 
    d) top 3 features values that contribute most to the positive evidence, 
    e) top 3 feature values that contribute the most to the negative evidence

In [181]:
testdata = X_test.as_matrix()

positive_lg_evidence=[]
negative_lg_evidence=[]
lg_evidence=[]
for i in testdata:
    
    positive_lg_evidence,negative_lg_evidence=evidence(i,ods_class_prior)

object_highest_PosEvi = test_data.iloc[(np.argmax(positive_lg_evidence))]
print("object with highest positive evidence:\n",object_highest_PosEvi )


c= Total_evidence(object_highest_PosEvi,ods_class_prior)
print("\ntotal positive evidence:",c[0])
print("\ntotal negative evidence:",c[1])
print("\nclass distributuion",(predict[np.argmax(positive_lg_evidence)]))

pos_evi=evidence(object_highest_PosEvi,ods_class_prior)[0]
neg_evi=evidence(object_highest_PosEvi,ods_class_prior)[1]

#print("\n top features",top_features(most_postive_object,len(pos_evi),len(neg_evi)))
top_features(object_highest_PosEvi,len(pos_evi),len(neg_evi))
indexNeg= top_features(object_highest_PosEvi,len(pos_evi),len(neg_evi))[0]
indexPos = top_features(object_highest_PosEvi,len(pos_evi),len(neg_evi))[1]

columns = X_test.columns

print("\n\ntop negative features are:",columns[indexNeg],"\ntop positive features",columns[indexPos])

object with highest positive evidence:
 f1     1
f2     1
f3     0
f4     0
f5     1
f6     1
f7     0
f8     0
f9     0
f10    1
f11    1
f12    0
f13    0
f14    0
f15    1
f16    1
f17    1
f18    0
f19    0
f20    1
f21    1
f22    0
Name: 0, dtype: int64

total positive evidence: 4.86266866212

total negative evidence: -4.01555564933

class distributuion [ 0.30003882  0.69996118]


top negative features are: Index(['f14', 'f22', 'f13'], dtype='object') 
top positive features Index(['f16', 'f5', 'f17'], dtype='object')


# 4. To find object with highest negative evidence


    a) the total positive log-evidence,
    b) the total negative log-evidence, 
    c) probability distribution, 
    d) top 3 features values that contribute most to the positive evidence, 
    e) top 3 feature values that contribute the most to the negative evidence

In [182]:
testdata = X_test.as_matrix()

positive_lg_evidence=[]
negative_lg_evidence=[]
lg_evidence=[]
for i in testdata:
    
    positive_lg_evidence,negative_lg_evidence=evidence(i,ods_class_prior)

object_highest_NegEvi = test_data.iloc[(np.argmax(negative_lg_evidence))]
print("object with highest Negative evidence:\n",object_highest_NegEvi )
d= Total_evidence(object_highest_NegEvi,ods_class_prior)
print("\ntotal positive evidence:",d[0],"\t\t total negative evidence:",d[1])
print("\nclass distributuion",(predict[np.argmax(negative_lg_evidence)]))



pos_evi=evidence(object_highest_NegEvi,ods_class_prior)[0]
neg_evi=evidence(object_highest_NegEvi,ods_class_prior)[1]

#print("\n top features",top_features(most_postive_object,len(pos_evi),len(neg_evi)))
top_features(object_highest_NegEvi,len(pos_evi),len(neg_evi))
indexNeg= top_features(object_highest_NegEvi,len(pos_evi),len(neg_evi))[0]
indexPos = top_features(object_highest_NegEvi,len(pos_evi),len(neg_evi))[1]

columns = X_test.columns

print("\n\ntop negative features are:",columns[indexNeg],"\ntop positive features",columns[indexPos])

object with highest Negative evidence:
 f1     1
f2     1
f3     0
f4     0
f5     1
f6     0
f7     0
f8     1
f9     1
f10    1
f11    1
f12    0
f13    1
f14    1
f15    1
f16    0
f17    1
f18    0
f19    0
f20    0
f21    1
f22    0
Name: 6, dtype: int64

total positive evidence: 7.18219836599 		 total negative evidence: -2.42725635682

class distributuion [ 0.00853556  0.99146444]


top negative features are: Index(['f22', 'f4', 'f19'], dtype='object') 
top positive features Index(['f8', 'f13', 'f17'], dtype='object')


# 5. To find object which is most uncertain  #fullon neend me kiya check again

    a) the total positive log-evidence,
    b) the total negative log-evidence, 
    c) probability distribution, 
    d) top 3 features values that contribute most to the positive evidence, 
    e) top 3 feature values that contribute the most to the negative evidence

In [186]:
# fully neend me kiya not sure kya krri hun

for i in predict:
    value= np.argmin(np.absolute([x[0]-x[1] for x in predict]))


print("object which most uncertain :\n", X_test.iloc[value])
e = Total_evidence(X_test.iloc[value],ods_class_prior)
print("\ntotal positive evidence:",e[0],"\t\t total negative evidence:",e[1])
print("\nclass distributuion",(predict[np.argmax(value)])) ### am not sure kya krri me


m,n=evidence(X_test.iloc[value],ods_class_prior)
print(m,n)

pos_evi=evidence(X_test.iloc[value],ods_class_prior)[0]
neg_evi=evidence(X_test.iloc[value],ods_class_prior)[1]

#print("\n top features",top_features(most_postive_object,len(pos_evi),len(neg_evi)))
top_features(X_test.iloc[value],len(pos_evi),len(neg_evi))
indexNeg= top_features(X_test.iloc[value],len(pos_evi),len(neg_evi))[0]
indexPos = top_features(X_test.iloc[value],len(pos_evi),len(neg_evi))[1]

columns = X_test.columns

print("\n\ntop negative features are:",columns[indexNeg],"\ntop positive features",columns[indexPos])


object which most uncertain :
 f1     1
f2     0
f3     0
f4     1
f5     0
f6     1
f7     0
f8     0
f9     0
f10    0
f11    1
f12    0
f13    0
f14    1
f15    0
f16    0
f17    0
f18    0
f19    0
f20    0
f21    0
f22    1
Name: 73, dtype: int64

total positive evidence: 3.6027223399 		 total negative evidence: -3.59483199074

class distributuion [ 0.30003882  0.69996118]
[0.59783700075562063, 0.78015855754957564, 0.057158413839949046, 0.11778303565638425, 0.82667857318446847, 0.044451762570833588, 1.1786549963416468] [-0.057158413839949108, -0.2029408439966903, -0.26662866325394874, -1.1102230246251565e-16, -0.42744401482693978, -0.075985906977922221, -0.090971778205726633, -0.1397619423751586, -0.44895022004790353, -0.1397619423751586, -0.080042707673536495, -0.32850406697203627, -0.16705408466316621, -0.26966356694910271, -0.16907633004393413]


top negative features are: Index(['f13', 'f8', 'f17'], dtype='object') 
top positive features Index(['f4', 'f14', 'f22'], dtype='obje